In [48]:
# make a nn using flax and jax to train on a simple XOR function
import jax
import jax.numpy as jnp
from flax import linen as nn
from jax import random, jit
import optax

In [49]:
devices = jax.devices()
print(devices)

[cuda(id=0)]


In [50]:
class SimpleNN(nn.Module):
    hidden_size: int

    @nn.compact
    def __call__(self, x):
        x = nn.Dense(self.hidden_size)(x)
        x = nn.relu(x)
        x = nn.Dense(self.hidden_size)(x)
        x = nn.relu(x)
        x = nn.Dense(self.hidden_size)(x)
        x = nn.relu(x)
        x = nn.Dense(self.hidden_size)(x)
        x = nn.relu(x)
        x = nn.Dense(self.hidden_size)(x)
        x = nn.relu(x)
        x = nn.Dense(self.hidden_size)(x)
        x = nn.relu(x)
        x = nn.Dense(5)(x)
        return x
    
    

def train_step(params, opt_state, x, y):
    grads = jax.grad(mse_loss)(params, model, x, y)
    updates, opt_state = optimizer.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    return params, opt_state

def mse_loss(params, model, x, y):
    preds = model.apply(params, x)
    loss = jnp.mean((preds - y) ** 2)
    return loss

def train(params, opt_state, x, y):
    for epoch in range(num_epochs):
        params, opt_state = train_step(params, opt_state, x, y)
        if epoch % 20 == 0:
            loss = mse_loss(params, model, x, y)
            print(f'Epoch {epoch}, Loss: {loss:.4f}')

            if loss < 0.0001:
                break
    return params, opt_state

In [51]:
# Hyperparameters
hidden_size = 25
input_size = 5

x = jnp.array(jnp.arange(0, input_size), dtype=jnp.float32)

y = jnp.array(5*x + 1, dtype=jnp.float32)
x

Array([0., 1., 2., 3., 4.], dtype=float32)

In [52]:
# Create the model
model = SimpleNN(hidden_size=hidden_size)
# Initialize parameters
rng = jax.random.PRNGKey(2)
params = model.init(rng, x)


In [53]:

# Create an optimizer
optimizer = optax.adam(learning_rate=0.01)
opt_state = optimizer.init(params)


In [54]:

# Perform a single training step
jit_train_step = jit(train_step)
params, opt_state = jit_train_step(params, opt_state, x, y)

# Training loop
num_epochs = 500


params, opt_state = train(params, opt_state, x, y)


Epoch 0, Loss: 167.2228
Epoch 20, Loss: 9.9519
Epoch 40, Loss: 0.6404
Epoch 60, Loss: 0.1009
Epoch 80, Loss: 0.0097
Epoch 100, Loss: 0.0002
Epoch 120, Loss: 0.0002
Epoch 140, Loss: 0.0000


In [55]:
x = jnp.array(jnp.arange(input_size,input_size+input_size), dtype=jnp.float32)
x



Array([5., 6., 7., 8., 9.], dtype=float32)

In [56]:


predictions = model.apply(params, x)
print("Predictions:", predictions)



Predictions: [ 2.4161375 15.922209  28.802181  42.306152  55.615265 ]
